In [3]:
# !pip install "opencv-python-headless<4.3"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 5.3 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.5.3.56
    Uninstalling opencv-python-headless-4.5.3.56:
      Successfully uninstalled opencv-python-headless-4.5.3.56


In [6]:
import numpy as np
from keras.models import load_model
import cv2
from keras.preprocessing import image
from tensorflow.keras.utils import img_to_array
# from keras.preprocessing.image import img_to_array
from time import sleep
import time

In [7]:
face_classifier = cv2.CascadeClassifier(r'haarcascade_frontalface_default.xml')
classifier =load_model(r'model.h5')


emotion_labels = ['Angry','Disgust','Fear','Happy','Neutral', 'Sad', 'Surprise']

cap = cv2.VideoCapture(0)
now = time.time()
future = now + 10
detected_emotions = []

while True:
    _, frame = cap.read()
    labels = []
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    faces = face_classifier.detectMultiScale(gray,scaleFactor=1.3, minNeighbors=5)

    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
        roi_gray = gray[y:y+h,x:x+w]
        roi_gray = cv2.resize(roi_gray,(48,48),interpolation=cv2.INTER_AREA)



        if np.sum([roi_gray])!=0:
            roi = roi_gray.astype('float')/255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi,axis=0)

            prediction = classifier.predict(roi)[0]
            label=emotion_labels[prediction.argmax()]
            label_position = (x,y-10)
            cv2.putText(frame,label,label_position,cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
            detected_emotions.append(label)
        else:
            cv2.putText(frame,'No Faces',(30,80),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
    cv2.imshow('Emotion Detector',cv2.resize(frame,(1600,960),interpolation = cv2.INTER_CUBIC))
    if cv2.waitKey(1) & 0xFF== ord('q'):
        break

    if time.time() > future:  ##after 10 second music will play
            cv2.destroyAllWindows()
            print("Music Player Opens", time)
            future = time.time() + 10
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 16ms/step
Music Player Opens <module 'time' (built-in)>
1/1 [==============================] - 0s 16ms/step
Music Player Opens <module 'time' (built-in)>
1/1 [==============================] - 0s 16ms/step
Music Player Opens <module 'time' (built-in)>
1/1 [==============================] - 0s 16ms/step
Music Player Opens <module 'time' (built-in)>
1/1 [==============================] - 0s 16ms/step
Music Player Opens <module 'time' (built-in)>
1/1 [==============================] - 0s 16ms/step


: 

In [3]:
detected_emotions

['Neutral',
 'Happy',
 'Neutral',
 'Happy',
 'Neutral',
 'Happy',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Happy',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Happy',
 'Happy',
 'Happy',
 'Happy',
 'Happy',
 'Happy',
 'Happy',
 'Happy',
 'Happy',
 'Happy',
 'Happy',
 'Happy',
 'Happy',
 'Happy',
 'Happy',
 'Happy',
 'Neutral',
 'Neutral',
 'Surprise',
 'Surprise',
 'Surprise',
 'Surprise',
 'Surprise',
 'Surprise',
 'Surprise',
 'Surprise',
 'Fear',
 'Surprise',
 'Surprise',
 'Fear',
 'Fear',
 'Happy',
 'Happy',
 'Happy',
 'Happy',
 'Surprise',
 'Surprise',
 'Fear',
 'Fear',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Sad',
 'Sad',
 'Neutral',
 'Neutral',
 'Neutral',
 'Sad',
 'Neutral',
 'Sad',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',